# **Notebook Setup: Import Libraries**

In [ ]:
# Libraries for Data Loading & Pre-processing
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Libraries for developing, training, and running GRU Neural Network
import tensorflow as tf

# Fallback api to fetch stock data
import yfinance

# Libraries for Metric Evaluation
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
)
from sklearn.preprocessing import MinMaxScaler

# **Setup: Parameter Adjustment**

In [ ]:
# Variables & Parameters to adjust below

# Data Loading
stock = "TSLA"
split = 0.8  # Test data is automatically calculated based on this

# Data Pre-processing
rw = 1  # Rollng Window or Input Window size

# Keras Layers Paramaters
# GRU Layer
units = 5
activation = "tanh"
# Dropout Layer
dropout = 0.2

# Compiling Parameters
loss = "mean_squared_error"
loss_scale_optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.01, momentum=0.9, nesterov=False
)

# Model Fitting Parameters
epochs = 3
batch_size = 1

# **Step 1: Data Loading**

In [ ]:
# Reused code from my prophet notebook, need to adopt for GRU specifically
# Todo, refactor this for GRU specifically
try:
    # Attempt to load data from local directory first
    df = pd.read_csv(f"../data/{stock}.csv")
    print(
        f"Successfully loaded data for {stock} from local directory '../data/'."
    )

except Exception as e:
    print(f"Failed to load data from local directory: {e}")
    print(f"Attempting to fetch data for {stock} using yfinance...")

    try:
        # Tries to load data from yfinance api since Local methood failed.
        stock_data = yfinance.Ticker(stock)
        df = stock_data.history(period="max")

        # yfinance uses 'Date' as index, reset to a column for consistency
        df = df.reset_index()
        df.rename(
            columns={
                "Date": "date",
                "Close": "close",
                "Volume": "volume",
                "Open": "open",
                "High": "high",
                "Low": "low",
            },
            inplace=True,
        )

        df2 = pd.DataFrame()
        df2["ds"] = df["date"].dt.date
        df2["y"] = df["close"]

        df = df2.copy()

        if not df.empty:
            print(f"Successfully fetched data for {stock} using yfinance.")
        else:
            print(
                "ERROR: No data could be loaded for analysis. Please check the ticker symbol matches yfinance"
            )

    except Exception as e:
        print(f"Failed to fetch data for {stock} using yfinance: {e}")
        print(
            "Both methods failed. Please ensure the data file is in '../data/'"
        )
        print(f"or that '{stock}' is a valid ticker symbol on yfinance.")
        df = None  # Ensure df is None if both methods fail

In [ ]:
# Displays the last 5 rows of the loaded dataset, check to make sure it's correct
if df is not None:
    print(df.tail())
else:
    print("Data could not be loaded using either method.")

# **Step 2: Data Pre-Processing**

In [ ]:
# Extract the date and closing price columns
dates = df["ds"]
closing_prices = df["y"]

# Calculate the number of data points for training and testing
number_of_data_points = len(df)
train_data_size = math.ceil(number_of_data_points * split)
print("Training Data Split Size:", train_data_size)

# Split the data into training and testing sets
train_data = pd.DataFrame(
    data={
        "Date": dates[:train_data_size],
        "Close": closing_prices[:train_data_size],
    }
)
test_data = pd.DataFrame(
    data={
        "Date": dates[train_data_size:],
        "Close": closing_prices[train_data_size:],
    }
)

# Print the first few rows of the training and testing data
print("Training data:")
print(train_data.head())
print("Training Data Shape", train_data.shape)

print("Testing data:")
print(test_data.head())
print("Testing Data Shape", test_data.shape)

In [ ]:
# Selecting Open Price values for Training Data
dataset_train = train_data.Close.values
print("OG Dataset_train 1 Dimensional:", dataset_train.shape)

# Reshaping 1D to 2D array for Training Data
dataset_train = np.reshape(dataset_train, (-1, 1))
print("New Dataset_train 2 Dimensional:", dataset_train.shape)


# Selecting Open Price values for Testing Data
dataset_test = test_data.Close.values
print("OG Dataset_test 1 Dimensional:", dataset_test.shape)

# Reshaping 1D to 2D array for Testing Data
dataset_test = np.reshape(dataset_test, (-1, 1))
print("New Dataset_test 2 Dimensional:", dataset_test.shape)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
# scaling both datasets, Normalizing values between 0 and 1
scaled_train = scaler.fit_transform(dataset_train)
scaled_test = scaler.fit_transform(dataset_test)

print("Training Dataset Normalized")
print(scaled_train[:5])
print("Normalized Training Dataset Shape:", scaled_train.shape)

print("Testing Dataset Normalized")
print(scaled_test[:5])
print("Normalized Testing Dataset Shape:", scaled_test.shape)

In [ ]:
# For training set
X_train = []
y_train = []

for i in range(rw, len(scaled_train)):
    X_train.append(scaled_train[i - rw : i, 0])
    y_train.append(scaled_train[i, 0])
    if i <= rw:
        print(
            "Training set,", rw, "Input Window Size to predict 1 Output Value"
        )
        print(X_train)
        print(y_train)
        print()

# For testing set
X_test = []
y_test = []
for i in range(rw, len(scaled_test)):
    X_test.append(scaled_test[i - rw : i, 0])
    y_test.append(scaled_test[i, 0])
    if i <= rw:
        print(
            "Testing set,", rw, "Input Window Size to predict 1 Output Value"
        )
        print(X_test)
        print(y_test)
        print()

In [ ]:
# The data is converted to Numpy array

# For training set
X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
y_train = np.reshape(y_train, (y_train.shape[0], 1))
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)


# For testing set
X_test = np.array(X_test)
y_test = np.array(y_test)

# Reshaping
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

# **Step 4: Building the Model**

In [ ]:
# Initialising the GRU model
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.GRU(
            units, input_shape=(X_train.shape[1], 1), activation=activation
        ),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(1, activation="relu"),
    ]
)

# Compiling GRU
model.compile(loss=loss, optimizer=loss_scale_optimizer)

# Fitting the data
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
model.summary()

# **Step 5: Make Forecast**

In [ ]:
# predictions with X_test data
y_GRU = model.predict(X_test)

In [ ]:
# scaling back from 0-1 to original, Un-Normalize the values
y_GRU_O = scaler.inverse_transform(y_GRU)
y_test_O = scaler.inverse_transform(y_test)

In [ ]:
# Verifying data shapes
print("Test Data Shape:", test_data.shape)
print("Predicted Data Shape:", y_GRU_O.shape)
print("y_test Data Shape:", y_test_O.shape)

# **Step 6: Visualizing Graphs**

In [ ]:
plt.style.use("fivethirtyeight")
fig, ax = plt.subplots()  # figsize=(18, 12)

# Plot for GRU predictions
ax.plot(train_data.index, train_data.Close, label="Training Data")
ax.plot(test_data.index, test_data.Close, label="Testing Data")
ax.plot(test_data.index[rw:], y_GRU_O, label="GRU Forecast")
ax.legend()
ax.title.set_text("GRU")

plt.xlabel("Days")
plt.ylabel("Share Price")

plt.show()

# **Step 7: Evaluate Model Accuracy**

In [ ]:
# calculate mean squared error
mse = mean_squared_error(y_test_O, y_GRU_O)
print("Mean Squared Error (MSE):", mse)

# calculate mean absolute error
mae = mean_absolute_error(y_test_O, y_GRU_O)
print("Mean Absolute Error (MAE):", mae)

# calculate root mean squared error
rmse = math.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# calculate mean absolute percentage error
mae = mean_absolute_percentage_error(y_test_O, y_GRU_O)
print("Mean Absolute Percentage Error (MAPE):", mae * 100)